In [ ]:
!pip install datasets py7zr

In [1]:
# Important: install these dependencies first:
# pip install datasets py7zr
from datasets import load_dataset
import os

In [2]:
os.environ["REPLICATE_API_TOKEN"] = "r8_XxUNPsB5I1iEJXo72xT2bhxqasc8oaQ3t1sOd"


In [3]:
PROMPT_TEMPLATE = "[INST] <<SYS>>\nUse the Input to provide a summary of a conversation.\n<</SYS>>\n\nInput:\n{message} [/INST]\n\nSummary: {summary}"


In [4]:
def format_instruction(sample):
    return {"text": PROMPT_TEMPLATE.format(message=sample["dialogue"], summary=sample["summary"])}


In [5]:
ds = load_dataset("samsum", split='train')
print(format_instruction(ds[5])['text'])



Reusing dataset samsum (/root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


[INST] <<SYS>>
Use the Input to provide a summary of a conversation.
<</SYS>>

Input:
Neville: Hi there, does anyone remember what date I got married on?
Don: Are you serious?
Neville: Dead serious. We're on vacation, and Tina's mad at me about something. I have a strange suspicion that this might have something to do with our wedding anniversary, but I have nowhere to check.
Wyatt: Hang on, I'll ask my wife.
Don: Haha, someone's in a lot of trouble :D
Wyatt: September 17. I hope you remember the year ;) [/INST]

Summary: Wyatt reminds Neville his wedding anniversary is on the 17th of September. Neville's wife is upset and it might be because Neville forgot about their anniversary.


In [6]:
ds = ds.map(format_instruction, remove_columns=['id', 'dialogue', 'summary'])
ds.to_json("samsum.jsonl")

Loading cached processed dataset at /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-0d383ff99652af18.arrow


Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

11387086

In [7]:
!cp samsum.jsonl data.jsonl

In [ ]:
%%bash

export REPLICATE_API_TOKEN="r8_QpQZBY0Jc4rJDxfNt3CRqaJgAQIzfOP2frFLj"

RESPONSE=$(curl -s -X POST -H "Authorization: Token $REPLICATE_API_TOKEN" https://dreambooth-api-experimental.replicate.com/v1/upload/data.jsonl)

curl -X PUT -H "Content-Type: application/jsonl" --upload-file data.jsonl "$(jq -r ".upload_url" <<< "$RESPONSE")"

SERVING_URL=$(jq -r ".serving_url" <<< $RESPONSE)
echo $SERVING_URL

In [ ]:
!ls

In [8]:
import replicate

In [ ]:
??replicate.trainings.create

In [10]:


training = replicate.trainings.create(
  version="meta/llama-2-7b:73001d654114dad81ec65da3b834e2f691af1e1526453189b7bf36fb3f32d0f9",
  input={
    "train_data": "https://replicate.delivery/pbxt/JwjWHjg1vStUyR6i69dbwi5FxmmDyxonF4WTWjFHK8Vrl4Rp/data.jsonl",
    "num_train_epochs": 3
  },
  destination="npatta01/llama2-samsum"
)

print(training)

id='zvtkd63bsmknurzxbzun4trbma' model='meta/llama-2-7b' version='73001d654114dad81ec65da3b834e2f691af1e1526453189b7bf36fb3f32d0f9' destination=None status='starting' input={'num_train_epochs': 3, 'train_data': 'https://replicate.delivery/pbxt/JwjWHjg1vStUyR6i69dbwi5FxmmDyxonF4WTWjFHK8Vrl4Rp/data.jsonl'} output=None logs='' error=None created_at='2023-11-26T21:28:57.367643556Z' started_at=None completed_at=None urls={'cancel': 'https://api.replicate.com/v1/predictions/zvtkd63bsmknurzxbzun4trbma/cancel', 'get': 'https://api.replicate.com/v1/predictions/zvtkd63bsmknurzxbzun4trbma'}


In [11]:
# If you've got a handle to the object returned by create()
training.reload()

# If you've got the training ID
training = replicate.trainings.get("zz4ibbonubfz7carwiefibzgga")

if training.status == "succeeded":
    print(training.output)
    # {"weights": "...", "version": "..."

ReplicateError: Not found.